In [1]:
import os
import cohere
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 환경 변수 로드
load_dotenv(override=True)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
# [필수] OpenAI 키 확인
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️ 경고: .env 파일에 OPENAI_API_KEY가 설정되지 않았습니다.")

# =========================================================
# 2. [전처리] 키워드 정규화 (Inline Injection 방식)
# =========================================================
def normalize_query(user_query):
    """
    사용자 쿼리 내의 일상 용어를 법률 용어로 변환하여,
    문맥 내에 '일상어(법률어)' 형태로 주입합니다.
    """
    keyword_dict = {
        # [주체]
        "집주인": "임대인", "주인": "임대인", "건물주": "임대인", "임대업자": "임대인",
        "세입자": "임차인", "월세입자": "임차인",
        "부동산": "공인중개사", "중개인": "공인중개사",
        
        # [금전]
        "보증금": "임차보증금", "전세금": "임차보증금", 
        "월세": "차임", "방세": "차임",
        "관리비": "관리비용",
        
        # [행위 - 돈]
        "올려": "증액", "인상": "증액", "더 달라고": "증액", "더 내라고": "증액",
        "깎아": "감액", "내려": "감액", "할인": "감액",
        "돌려주지": "반환", "안 줘": "반환거부", "떼먹": "미반환",
        
        # [행위 - 거주/계약]
        "계약서": "임대차계약증서", "특약": "특약사항",
        "이사": "주택의인도", "전입": "주민등록/대항력",
        "살고": "점유", "거주": "점유",
        "나가라": "계약갱신거절/명도", "나가라고": "계약갱신거절/명도", "비워": "명도", "쫓겨": "명도/대항력",
        "방 빼": "계약해지", "나갈게": "계약해지", "해지": "계약해지",
        "연장": "계약갱신", "더 살": "계약갱신", "재계약": "계약갱신",
        "수리": "수선의무/필요비", "고쳐": "수선의무", "물 새": "누수", "곰팡이": "하자",
        "청소비": "원상회복비용", "원상복구": "원상회복"
    }
    
    normalized = user_query
    # 긴 단어부터 매칭되도록 정렬
    sorted_keys = sorted(keyword_dict.keys(), key=len, reverse=True)
    
    for key in sorted_keys:
        if key in normalized:
            legal_term = keyword_dict[key]
            target = f"{key}({legal_term})"
            if target not in normalized:
                normalized = normalized.replace(key, target)
                
    return normalized

# =========================================================
# 3. [검색 & Rerank] 하이브리드 검색
# =========================================================
def legal_hybrid_retrieval(query, pinecone_store, k_fetch=50, k_final=5, score_threshold=0.25):
    # k_fetch를 50으로 늘려 민법 조항(원상회복 등)이 포함될 확률을 높임
    print(f"🔍 [1차 검색] Pinecone 탐색: '{query}'")
    try:
        initial_docs = pinecone_store.similarity_search(query, k=k_fetch)
    except Exception as e:
        print(f"❌ 검색 실패: {e}")
        return []
    
    if not initial_docs:
        return []

    # Cohere Rerank
    docs_content = [doc.page_content for doc in initial_docs]
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    try:
        rerank_results = co.rerank(
            model="rerank-multilingual-v3.0",
            query=query,
            documents=docs_content,
            top_n=k_fetch
        )
    except Exception as e:
        print(f"⚠️ Rerank 실패: {e}")
        return initial_docs[:k_final]

    relevant_docs = []
    for r in rerank_results.results:
        if r.relevance_score < score_threshold:
            continue
        doc = initial_docs[r.index]
        doc.metadata['relevance_score'] = r.relevance_score
        relevant_docs.append(doc)

    final_docs = sorted(
        relevant_docs, 
        key=lambda x: int(x.metadata.get('priority', 99))
    )
    
    return final_docs[:k_final]

# =========================================================
# 4. [포맷팅]
# =========================================================
def format_legal_docs(docs):
    display_html = '<div class="legal-docs-container mb-3">'
    llm_context = ""
    for i, doc in enumerate(docs):
        title = doc.metadata.get('title', '')
        # 이중 줄바꿈 제거
        content = doc.page_content.strip().replace("\n\n", "\n")
        src_title = doc.metadata.get('src_title', '')
        priority = doc.metadata.get('priority', 99)
        
        llm_context += f"참고문서 {i+1}.\n출처: {src_title}\n조항: {title}\n내용: {content}\n\n"
        
        badge_color = "bg-danger" if priority == 1 else "bg-secondary"
        display_html += f"""
        <div class="card mb-2 shadow-sm">
            <div class="card-header d-flex justify-content-between align-items-center py-2">
                <span class="fw-bold text-dark">{title}</span>
                <span class="badge {badge_color}">{src_title}</span>
            </div>
            <div class="card-body py-2 bg-light">
                <p class="card-text text-secondary" style="white-space: pre-line; font-size: 0.9rem;">{content}</p>
            </div>
        </div>"""
    display_html += '</div>'
    return {"display_html": display_html, "llm_context": llm_context}

# =========================================================
# 5. [생성] LLM 답변 생성 (GPT-5 Nano 적용)
# =========================================================
def generate_legal_advice(normalized_query, context_text):
    print(f"🤖 [LLM 입력] Query: {normalized_query}")
    print(f"⏳ GPT-5 Nano가 답변을 생성 중입니다...")
    
    system_prompt = """
    당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
    사용자(임차인)가 이해하기 쉽도록 **현대적인 한국어(존댓말)**로 답변하세요.

    [지시사항]
    1. **근거 기반**: 반드시 [참고 법령]의 내용을 토대로 답변하세요.
    2. **결론 우선**: 가능/불가능 여부를 두괄식으로 명확히 제시하세요.
    3. **위험 경고**: 임차인에게 불리한 특약이나 강행규정 위반 사항은 반드시 [주의사항]으로 경고하세요.
    4. **면책**: 법적 효력이 없는 참고용임을 밝히세요.

    [참고 법령]
    {context}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "사용자 입력: {question}"),
    ])

    # [핵심 변경] OpenAI GPT-5 Nano 적용
    llm = ChatOpenAI(
        model="gpt-5-mini",
        temperature=0,       # 법률 답변의 정합성을 위해 0 설정
        openai_api_key=OPENAI_API_KEY
    )

    chain = prompt | llm | StrOutputParser()
    
    try:
        return chain.invoke({"context": context_text, "question": normalized_query})
    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"

c:\Users\Admin\Desktop\Project2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Pinecone 연결 (기존 객체 사용 가능 시 생략)
embedding_model = UpstageEmbeddings(model="solar-embedding-1-large-passage")
vector_store = PineconeVectorStore(
    pinecone_api_key=PINECONE_API_KEY,
    index_name="law-index",
    embedding=embedding_model
)

In [3]:
# ---------------------------------------------------------
# ▼▼▼ 테스트할 질문을 여기에 입력하세요 ▼▼▼
# user_input = "집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?"
user_input = "계약서의 이 조항 괜찮을까? '특약사항: 임차인은 퇴실 시 청소비 30만원을 무조건 공제한다.'"
# ---------------------------------------------------------

# 1. 정규화
norm_query = normalize_query(user_input)
print(f"🔄 [Step 2] 정규화된 쿼리: {norm_query}")

# 2. 검색 (k_fetch=50으로 민법 탐색 강화)
retrieved_docs = legal_hybrid_retrieval(
    query=norm_query, 
    pinecone_store=vector_store,
    k_fetch=50,
    k_final=5,
    score_threshold=0.25
)

if retrieved_docs:
    print(f"\n✅ [Step 3] 관련 법령 {len(retrieved_docs)}건 확보")
    formatted_data = format_legal_docs(retrieved_docs)
    
    # 3. GPT-5 Nano 생성
    final_answer = generate_legal_advice(
        normalized_query=norm_query, 
        context_text=formatted_data['llm_context']
    )
    
    print("\n" + "="*50)
    print("🤖 GPT-5 Mini 답변")
    print("="*50)
    print(final_answer)
else:
    print("⚠️ 검색 결과가 없습니다.")

🔄 [Step 2] 정규화된 쿼리: 계약서(임대차계약증서)의 이 조항 괜찮을까? '특약(특약사항)사항: 임차인은 퇴실 시 청소비(원상회복비용) 30만원을 무조건 공제한다.'
🔍 [1차 검색] Pinecone 탐색: '계약서(임대차계약증서)의 이 조항 괜찮을까? '특약(특약사항)사항: 임차인은 퇴실 시 청소비(원상회복비용) 30만원을 무조건 공제한다.''

✅ [Step 3] 관련 법령 3건 확보
🤖 [LLM 입력] Query: 계약서(임대차계약증서)의 이 조항 괜찮을까? '특약(특약사항)사항: 임차인은 퇴실 시 청소비(원상회복비용) 30만원을 무조건 공제한다.'
⏳ GPT-5 Nano가 답변을 생성 중입니다...

🤖 GPT-5 Nano 답변
결론(두괄식)
- 가능성: 해당 특약은 원칙적으로 당사자 간 합의로 계약서에 넣을 수는 있으나, "임차인에게 일방적으로 불리하게 고정액을 무조건 공제"하도록 한 문구는 분쟁 발생 시 법원이 무효·감액 판단을 할 수 있어 권장하지 않습니다.
- 권장사항: 실제 발생비용에 따른 정산, 증빙(영수증·사진) 제출, 퇴거 전 점검 절차 등을 포함하도록 특약문구를 수정하시길 권합니다.

근거(참고 법령에 따른 요지)
- 주택임대차계약은 서면체결 시 표준계약서를 우선 사용하도록 규정되어 있습니다. 당사자가 다른 서식을 사용하기로 합의하면 가능하나, 표준계약서 사용을 우선 고려해야 합니다(주택임대차보호법 제30조).  
- 임차인이 보증금을 회수하려면 임차주택을 인도(퇴거)해야 한다는 규정이 있으므로, 보증금과 관련한 정산은 인도 이후의 처리·증빙에 따라 이루어져야 하는 점을 유의해야 합니다(주택임대차보호법 제3조의2 제3항).

해당 특약의 문제점(주의사항)
1. 객관적 비용과 무관한 "무조건 공제"는 임차인에게 과도한 불이익을 주는 것으로 판단될 소지가 큽니다. 법원은 실제 손해액·통상적인 원상회복 비용과 비교하여 공제의 타당성을 심사합니다.  
2. 보증금에서 일부를 임의로 공제하려면 그 근거(손해·오염·수리비 등)와 비용 증빙이

In [4]:
# ---------------------------------------------------------
# ▼▼▼ 테스트할 질문을 여기에 입력하세요 ▼▼▼
user_input = "집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?"
# user_input = "계약서의 이 조항 괜찮을까? '특약사항: 임차인은 퇴실 시 청소비 30만원을 무조건 공제한다.'"
# ---------------------------------------------------------

# 1. 정규화
norm_query = normalize_query(user_input)
print(f"🔄 [Step 2] 정규화된 쿼리: {norm_query}")

# 2. 검색 (k_fetch=50으로 민법 탐색 강화)
retrieved_docs = legal_hybrid_retrieval(
    query=norm_query, 
    pinecone_store=vector_store,
    k_fetch=50,
    k_final=5,
    score_threshold=0.25
)

if retrieved_docs:
    print(f"\n✅ [Step 3] 관련 법령 {len(retrieved_docs)}건 확보")
    formatted_data = format_legal_docs(retrieved_docs)
    
    # 3. GPT-5 Nano 생성
    final_answer = generate_legal_advice(
        normalized_query=norm_query, 
        context_text=formatted_data['llm_context']
    )
    
    print("\n" + "="*50)
    print("🤖 GPT-5 Mini 답변")
    print("="*50)
    print(final_answer)
else:
    print("⚠️ 검색 결과가 없습니다.")

🔄 [Step 2] 정규화된 쿼리: 집주인(임대인)이 월세(차임)를 올려(증액) 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라(계약갱신거절/명도)고(계약갱신거절/명도) 하네요. 계약서(임대차계약증서)에는 그런 얘기가 없는데 이렇게 쫓겨(명도/대항력)나도 되나요?
🔍 [1차 검색] Pinecone 탐색: '집주인(임대인)이 월세(차임)를 올려(증액) 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라(계약갱신거절/명도)고(계약갱신거절/명도) 하네요. 계약서(임대차계약증서)에는 그런 얘기가 없는데 이렇게 쫓겨(명도/대항력)나도 되나요?'

✅ [Step 3] 관련 법령 4건 확보
🤖 [LLM 입력] Query: 집주인(임대인)이 월세(차임)를 올려(증액) 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라(계약갱신거절/명도)고(계약갱신거절/명도) 하네요. 계약서(임대차계약증서)에는 그런 얘기가 없는데 이렇게 쫓겨(명도/대항력)나도 되나요?
⏳ GPT-5 Nano가 답변을 생성 중입니다...

🤖 GPT-5 Mini 답변
요약(결론) — 두괄식
- 결론: 임대인이 “월세 올리면 나가라” 하고 바로 쫓아내는 것은 정당한 절차가 아닙니다. 임대인은 법률(주택임대차보호법 제7조 등)에 따라 일정한 범위 내에서만 증액을 청구할 수 있고, 임차인은 법정 한도를 초과한 금액을 이미 지급했거나 지급하는 경우 반환을 청구할 수 있습니다(주택임대차보호법 제7조, 제10조의2, 시행령 제8조, 민법 제628조). 임대인이 즉시 강제퇴거(자력집행)를 할 수는 없고, 퇴거를 원한다면 통상적으로 법적 절차를 밟아야 합니다.

근거(관련 법조문 요약)
- 주택임대차보호법 제7조: 임대인·임차인은 조세·공과금 등 부담 증감·경제사정 변동으로 약정 차임·보증금이 적절하지 않으면 장래에 대해 증감청구할 수 있음. 다만 증액청구는 최근 증액·약정 후 1년 이내에는 하지 못함.
- 주택임대차보호법 시행령 제8조: 증액청구는 약정 차임 등의 1/20(5%)를 초과할 수 없음. (